# HARVESTING API PIHPS

# Import Library

In [1]:
#import seluruh library yang akan digunakan
import psycopg2
import pandas as pd
import json
import requests
import keyring
from datetime import timedelta, date
import itertools
from sqlalchemy import create_engine

# Import API

In [2]:
#persiapan loop dan headers API
province_id = ['6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','81','83','84']
headers = {
    "Authorization": "0bd7b7f5164e6497543c93db6080fa5ff90746467fc915438e6004524a86b743ae91abbe7bd376f4bcda49685c78ff52d122b00cc3437f0af182685d5d24aabdrhSH6PH2pSOdOtLsaNZV8SeQFrfLwmVbMCbmDqVBijQ8Q2Nlni7eWioxBbzo+LPg"
}

In [60]:
#loop API request dan simpan hasil pada responses1 
responses1 = []
tanggal = []

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2018, 3, 16)
end_dt = date(2018, 3, 31)
#end_dt = date.today()
for dt in daterange(start_dt, end_dt):
    tanggal.append(dt.strftime("%Y-%m-%d"))

for x,y in list(itertools.product(province_id, tanggal)):
   req1 = requests.get('http://api.bappenas.go.id/bus/api/domain/newpihps/getIntegrationprices?province_id={}'.format(x)+'&period={}'.format(y), headers=headers).json()
   #tambahkan hasil ke dalam responses1
   responses1.append(req1)

# Pembentukan Dataframe

In [61]:
#df1 = pd.json_normalize(data = responses1, record_path = 'data')
#result = df1.to_json(orient="table")
#parsed = json.loads(result)
#df1 = pd.json_normalize(data = parsed['data'], record_path = 'details', meta =['market_id', 'date', 'inputted', 'validated'])
#df1 = df1[['date','inputted','validated','market_id','commodity_id','quality_id','price']]
#df1.index = df1.index + 1
#df1['market_id'] = df1['market_id'].astype('int64')
#df1

df1 = pd.json_normalize(data = responses1)
result = df1.to_json(orient="table")
parsed = json.loads(result)
df2 = pd.json_normalize(data = parsed['data'], record_path = 'data')
result1 = df2.to_json(orient="table")
parsed1 = json.loads(result1)
df3 = pd.json_normalize(data = parsed1['data'], record_path = 'details', meta =['market_id', 'date', 'inputted', 'validated'])
df3 = df3[['date','inputted','validated','market_id','commodity_id','quality_id','price']]
df3.index = df3.index + 1
df3['market_id'] = df3['market_id'].astype('int64')
df3

,date,inputted,validated,market_id,commodity_id,quality_id,price
1,2018-03-16,2018-03-16 02:30:58,2018-03-16 03:00:16,140,6,16,25000
2,2018-03-16,2018-03-16 02:30:58,2018-03-16 03:00:16,140,9,8,18000
3,2018-03-16,2018-03-16 02:30:58,2018-03-16 03:00:16,140,4,18,48000
4,2018-03-16,2018-03-16 02:30:58,2018-03-16 03:00:16,140,3,1,10300
5,2018-03-16,2018-03-16 02:30:58,2018-03-16 03:00:16,140,3,5,9300
...,...,...,...,...,...,...,...
59590,2018-03-29,2018-03-29 02:49:13,2018-03-29 03:44:34,156,4,17,32000
59591,2018-03-29,2018-03-29 02:49:13,2018-03-29 03:44:34,156,3,5,11000
59592,2018-03-29,2018-03-29 02:49:13,2018-03-29 03:44:34,156,2,20,28000
59593,2018-03-29,2018-03-29 02:49:13,2018-03-29 03:44:34,156,5,88,38000


# Ekspor ke DB

In [7]:
#set koneksi ke database
uri = 'postgresql://pusdatin:'+keyring.get_password("tim2", "pusdatin")+'@10.1.178.88:5432/pusdatin'
engine = create_engine(uri)

In [62]:
#ekspor dataframe harga
#df1.to_sql('data', con = engine, schema = 'pihps', if_exists='append')
df3.to_sql('data', con = engine, schema = 'pihps', if_exists='append')